In [ ]:
from math import sqrt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import datetime
import seaborn as sns
from scipy import stats

In [ ]:
def scale(dataset,col_name, output_array=False):
    """Standard Z score scaler"""
    array = dataset[col_name].to_numpy()
    mean = np.mean(array)
    std = np.std(array)
    scaled = (array-mean)/std
    if output_array:
        return scaled
    else:
        return scaled, mean, std 

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, variable_num = 0,  dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Sequence of observations as a list or NumPy array.
    n_in: Number of lag observations as input (X).
    n_out: Number of observations as output (y).
    dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names+=[('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[variable_num].shift(-i))
        if i == 0:
            names.append('var1(t)')
        else:
            names.append('var%d(t+%d)' % (1, i))
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg, names

In [ ]:
def random_date(array, lockdown=False):
    """ Choose a random date within lockdown or out of lockdown."""
    if lockdown:
        return np.random.choice(array[(array > pd.to_datetime('2020-03-24')) & (array < pd.to_datetime('2020-06-23')) ])
    else:
        #ensure a years worth of hourly data is available for training.
        array = pd.to_datetime(array[(24*365):])
        
        # 2020-01-01 ensures only non-lockdown entries will be considered
        return np.random.choice(array[array < pd.to_datetime('2020-01-01')])

In [ ]:
def split_by_date( dataset , reframed_data, date, days_prior=60):
    
    """ Split reframed data by specific date, days refers to the number 
    of days after the specified date the test set will contain.
    days prior refers to the number of days used to train the model """
    
    #get values from reframed data
    values = reframed_data.values
    
    #convert string into datetime
    date = pd.to_datetime(date)
    
    #create new dataframe
    df = dataset.reset_index().rename(columns = {'index':'date'})
    
    #ensure dates are in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    #find index for date
    date1_index = df.index[df['date'] == date1 ][0]
    
    #set train values to be all data upto given date 
    train = values[ date1_index - n_hours - (24*days_prior) : date1_index - n_hours, :]
    
    #remove training dataset so that tests can be done on the remaining datapoints 
    leftover_before = values[ : date1_index - n_hours - (24 * days_prior) , :]
    leftover_after = values[ date1_index - n_hours : , :]

    #get index of second date is 1 month into future
    date2_index = date1_index + (30 * 24)

    #test set two months after specified date
    test = values[ ( date1_index - n_hours ) : (date2_index - n_hours)  + 1 , :]
    
    #get specific dates for test period
    dates = df.date.to_numpy()
    specific_dates = dates[ date1_index  : date2_index+1  ]
    
    if leftover_before.shape[0] > leftover_after.shape[0]:
        
        return train , test, specific_dates, leftover_before
    
    else:
        return train , test, specific_dates, leftover_after

In [ ]:
def reshape(data, n_features):
    """ Reshape puts data in apprpriate format for LSTM usage
    [samples, timesteps, features] """
    n_obs = n_hours * n_features

    #split according to lag columns and output
    data_X, data_y = data[:, :n_obs], data[:, -hours_after:]
         
    # reshape input to be 3D [samples, timesteps, features]
    data_X_r = data_X.reshape((data_X.shape[0], n_hours , n_features))

    return data_X, data_y , data_X_r 

In [ ]:
def inverse_scale(array,mean,std):
    """ Inversion of Standard scaler """
    inverse = (array*std) + mean 
    return inverse

In [ ]:
def LSTM_model(cells,dropout):
    # design network
    model = Sequential()
    model.add(LSTM(cells, activation = 'relu',dropout=dropout,input_shape=(train_X_r.shape[1], train_X_r.shape[2])))
    model.add(Dense(n_hours, activation = 'relu'))
    model.add(Dense(n_hours_out))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

In [ ]:
def hour_predictor(test_r, index , timesteps):
    """ Rolling one hour prediction: uses previous 
    1 hour predictions to predict 1 hour ahead """
    
    #starting sequence used to get first prediction. 
    start = test_r[index].reshape(1,n_hours,n_features)
    
    #empty array to store results 
    results = []
    
    #get first prediction
    result = model.predict(start)[0][0]
    
    #record it 
    results.append(result)
    
    for i in range (1,timesteps):

        #shift window one time step
        start = np.delete(start[0],0,0)
        next_ts = test_r[index+i][2]
        start = np.append(start,next_ts).reshape(1,n_hours,n_features)

        #add prediction
        start[0][n_hours-1][0] = result

        #make new prediction
        result = model.predict(start)[0][0]

        results.append(result)
        
    return np.array(results)

In [ ]:
# load dataset

dataset = pd.read_csv('./MCC_combined.csv', header=0, index_col=0)

#conver days column into one-hot-encoded variable
days =['Mon','Tue','Wed','Thurs','Fri','Sat','Sun']
data = pd.get_dummies(dataset['day_of_week'])
data.columns = days
dataset  = dataset.join(data)

#get all dates
all_dates = pd.to_datetime(dataset.index.to_numpy())


#scale continuous variables --> ['temp', 'ws', 'wd', 'Volume']
continuous_vars = ['temp', 'ws', 'wd', 'Volume']
for i in continuous_vars:
    dataset[i] = scale(dataset,i,output_array=True)
    
# scale NO2 keep mean and std for inverse scaling
no2, mean, std, = scale(dataset,'NO2',output_array=False)
dataset['NO2'] = no2

#chose variables

#-----NO2 only 
# ['NO2']

#-----NO2 + traffic volume

#['NO2','Volume']


#------NO2 + meteorological

#['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4']


#------ ALL VARIABLES

#['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
       #'Thurs', 'Fri', 'Sat', 'Sun']


dataset.head()

In [ ]:
# paramter ranges
HOURS_BEFORE = [24]
BATCH_SIZE = [64]
CELLS = [24]
DROPOUT= [0]
EPOCHS = [50]
TRAIN_SIZE = [30]
VARIABLES = [['NO2'],['NO2','Volume'], 
            ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4'],
            ['NO2', 'ws' ,'temp','wd_1', 'wd_2', 'wd_3', 'wd_4', 'Volume','Mon', 'Tue', 'Wed',
             'Thurs', 'Fri', 'Sat', 'Sun']]
TIMESTEPS = 1 # for rolling prediction
LOCKDOWN = False
rdm_date = False
plot_days = 30


if rdm_date:
    #choose a random date 
    DATE = random_date(all_dates,lockdown=LOCKDOWN)
else:
    DATE =  '2020-03-24' #First lockdown
    DATE1 = '2020-02-24' #Month Before Lockdown
    
results = []

print ('runs:', len(HOURS_BEFORE)*len(BATCH_SIZE)
      *len(CELLS)* len(DROPOUT) * len(EPOCHS) * len(TRAIN_SIZE)
      * len(VARIABLES)* 3)

In [ ]:
# #create new dataframe
# df = dataset.reset_index().rename(columns = {'index':'date'})

# #ensure dates are in datetime format
# df['date'] = pd.to_datetime(df['date'])

# #find index for date
# df[df['date'] >= DATE ]

In [ ]:
#-----------UPDATE PREDICTOR-----------
count = 0 
#---specify bounds of lag and prediction
for h in HOURS_BEFORE:
    for b in BATCH_SIZE:
        for c in CELLS:
            for d in DROPOUT:
                for e in EPOCHS:
                    for t in TRAIN_SIZE:
                        for v in VARIABLES:
                            
                            print (f"hb:{h}, ts:{t}, e:{e}, bs:{b}, c:{c}, vars:{len(v)}  ")
                            
                            dataset_copy = dataset[v]
                            
                            hours_before = h
                            hours_after = TIMESTEPS

                            days_before = 1
                            days_after = 1

                            # # specify the number of lag hours
                            n_hours = hours_before * days_before # lag hours for each feature (t-x)
                            n_features = len(v) # number of features 
                            n_hours_out = hours_after * days_after # lenghth of output forecast (<1 == <t+1)

                            date1 = DATE

                            #convert values
                            values = dataset_copy.values.astype('float32') 

                            # frame as supervised learning
                            reframed, names = series_to_supervised(values, n_hours, n_hours_out )


                            train , test,  dates , leftover = split_by_date(dataset_copy, reframed, date1, t)

#                             print(train.shape, test.shape, dates.shape)

                            train_X, train_y , train_X_r = reshape(train,n_features)
                            test_X, test_y , test_X_r = reshape(test,n_features)

#                             print( train_X_r.shape, test_X_r.shape)

                            for rep in range (3):

                                model = LSTM_model(c,d)

                                # fit network
                                history = model.fit(train_X_r, train_y, epochs=e, batch_size=b, validation_split = 0.2 , verbose=0, shuffle=False)

                                # get scaled prediction and observed values to calculate RMSE
                                predicted = model.predict(test_X_r)[::n_hours_out]
                                observed = test_y[::n_hours_out]

                                #calculate RMSE
                                RMSE = mean_squared_error(observed, predicted,squared = False)

                                # rescale for plotting
                                predicted_rescaled = inverse_scale(predicted.flatten(), mean, std)[:dates.shape[0]]
                                observed_rescaled = inverse_scale(observed.flatten(), mean, std)[:dates.shape[0]]
                                
                                MAPE = mean_absolute_percentage_error(observed_rescaled,predicted_rescaled)


                            #     #calculate mean RMSE for each timestep 
                            #     lag_scores = np.array([mean_squared_error(observed_update[i],predicted_update[i], squared=False ) for i in range(observed_update.shape[0])])
                            #     lag_scores = np.mean(lag_scores.reshape(int(observed_update.shape[0]/TIMESTEPS),TIMESTEPS),axis=0)

                                                  
                                results.append([h,t,e, b, c, len(v) ,RMSE, MAPE] )
                                count +=1 
                                print(f'{count}, {rep}, RMSE:{RMSE}, MAPE: {MAPE} ')
                                #plot 
                                if rep == 2: 
                                    plt.figure(figsize = (15,8))
                                    plt.plot(dates[:24*plot_days],observed_rescaled[:24*plot_days],label='Observed')
                                    plt.plot(dates[:24*plot_days],predicted_rescaled[:24*plot_days],label='Predicted')
                                    plt.legend()
                                    plt.xticks(rotation=45)
                                    plt.show()

In [ ]:
results

In [ ]:
# # ----- ROLLING 1 HOUR PREDICTOR---------
# TIMESTEPS = 3

# #specify bounds of lag and prediction
# hours_before = 24
# hours_after = 1

# days_before = 1
# days_after = 1

# # # specify the number of lag hours
# n_hours = hours_before * days_before # lag hours for each feature (t-x)
# n_features = len(dataset.columns) # number of features 
# n_hours_out = hours_after * days_after # lenghth of output forecast (<1 == <t+1)

# date1 = DATE

# #convert values
# values = dataset.values.astype('float32') 

# # frame as supervised learning
# reframed, names = series_to_supervised(values, n_hours, n_hours_out )

# print(f'1h rolling Model will use last {n_hours} hours from {list(dataset.columns)} variables to predict {TIMESTEPS} ahead.')

# train , test,  dates , leftover = split_by_date( dataset, reframed, date1, TRAIN_SIZE )


# print ( dates.shape, test.shape ) 

# #print(train.shape, test.shape, dates.shape)

# train_X, train_y , train_X_r = reshape(train,n_features)
# test_X, test_y , test_X_r = reshape(test,n_features)

# print( train_X_r.shape, test_X_r.shape)


# for rep in range (1):

#     model = LSTM_model(CELLS,DROPOUT)

#     # fit network
#     history = model.fit(train_X_r, train_y, epochs = EPOCHS, batch_size=BATCH_SIZE, validation_split = 0.2 , verbose=0, shuffle=False)

#     #timesteps = how far ahead the rolling 1 hour predictor will forcast. 
#     # array of indexes  
#     indexes = np.arange(0,test_X_r.shape[0] - TIMESTEPS , TIMESTEPS)

#     # use rolling hour predictor to predict x timesteps ahead at each index within the array to get 
#     # contiguous predictions over a longer timeframe.
#     predicted = np.array([hour_predictor(test_X_r, i,  TIMESTEPS) for i in indexes]).flatten()
    
#     observed = test_y.flatten()[:predicted.shape[0]]

#     #invert scale and reshape for evaluation
#     predicted_rescaled = inverse_scale( predicted.reshape(len(predicted),1), mean, std )

#     #invert scale observed values, trim observed values so that predicted and observed have equal sizes and shape
#     observed_rescaled = inverse_scale(test_y.flatten()[:predicted.shape[0]],mean,std).reshape(predicted.shape[0],1)

#     #calculate RMSE
#     total_score = mean_squared_error(observed,predicted,squared = False)

# #     #calculate mean RMSE for each timestep 
# #     lag_scores = np.array([mean_squared_error(observed[i],predicted[i], squared=False ) for i in range(observed.shape[0])])
# #     lag_scores = np.mean(lag_scores.reshape(int(observed.shape[0]/TIMESTEPS),TIMESTEPS),axis=0)

# #     print (f"Rolling Predictor: Epochs: {EPOCHS}, Batch Size: {BATCH_SIZE}, TIMESTEP:{TIMESTEPS}, RMSE:{total_score}, Lag Scores {lag_scores}")
    
#     print ( total_score )
#     #trim dates 
#     #dates = dates[:predicted.shape[0]]
#     plt.figure(figsize = (15,8))
#     plt.plot(dates[:24*plot_days],observed_rescaled[:24*plot_days],label='Observed')
#     plt.plot(dates[:24*plot_days],predicted_rescaled[:24*plot_days],label='Predicted')
#     plt.legend()
#     plt.xticks(rotation=45)
#     plt.show()

In [ ]:
# def averager(array):
#     before = 0
#     after = array.shape[0]-1
#     results = []
#     for i in range(array.shape[0]):

#         before_array = np.zeros(before)
#         before_array[:] = np.nan

#         after_array = np.zeros(after)
#         after_array[:] = np.nan

#         results.append(np.concatenate((before_array, array[i], after_array)))

#         before = before + 1  
#         after= after - 1 

#     results = np.vstack(results)
    
#     return np.nanmean(results,axis=0), np.nanstd(results,axis=0)